# Assignment 1
## Shuai Shao

In [1]:
from feat import Detector
from feat.plotting import imshow
from IPython.display import Image
import pandas as pd
import cv2
from pathlib import Path

In [33]:
df = pd.read_csv("../dataset/annotations.csv")
#df["path"] = list(map(lambda x: x.join(".jpg"), list(df.file)))
print(df)

                file   valence
0          thumbs-up  positive
1            sad-man  negative
2              claws  negative
3    laughing-couple  positive
4         by-the-sea  negative
5            arguing  negative
6   enjoying-the-sun  positive
7          piggyback  positive
8           back-off  negative
9          handshake  positive
10            tablet  negative
11         happy-man  positive
12         sad-woman  negative
13          business  positive
14              pain  negative
15          students  positive
16             bills  negative
17           bullied  negative
18      disagreement  negative
19      grandparents  positive


In [8]:
detector = Detector(face_model="retinaface", au_model = "xgb", emotion_model="resmasknet")






In [49]:

all_aus = pd.DataFrame()



def proc_image(path, detector, all_aus):

    pred = detector.detect_image(path)

    aus = pred.aus

    name = Path(path).stem
    aus['file'] = name
    aus['face'] = range(len(aus))
    print(len(all_aus))
    if len(all_aus) < 1:
        all_aus = aus
    else:
        all_aus.append(pd.DataFrame(aus))
    #print(all_aus)

    fb = pred.faceboxes

    #return all_aus

[*map(lambda y: proc_image(y, detector, all_aus), map(lambda x: f"../dataset/images/{x}.jpg", df['file']))]

print(all_aus)


#aus.to_csv("processed/" + name + ".csv", index_label = 'face')


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.19s/it]
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\1883044432.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['file'] = name
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\1883044432.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['face'] = range(len(aus))


0


100%|██████████| 1/1 [00:01<00:00,  1.97s/it]
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\1883044432.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['file'] = name
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\1883044432.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['face'] = range(len(aus))


0


...

100%|██████████| 1/1 [00:03<00:00,  3.07s/it]
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\1883044432.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['file'] = name
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\1883044432.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['face'] = range(len(aus))


0


100%|██████████| 1/1 [00:03<00:00,  3.79s/it]

0
Empty DataFrame
Columns: []
Index: []



C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\1883044432.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['file'] = name
C:\Users\Shuai Shao\AppData\Local\Temp\ipykernel_22600\1883044432.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aus['face'] = range(len(aus))


## Reflections
### 1.


### 2.
* I would use backward feature elimination to determine which features yields the best performance increase / lowest performance decrease at the specified number of features. While it may be tempting to choose the AUs with the greatest mean difference between classes, these AUs may be intercorrelated, meaning that using more than one would may provide little to new information gain.
* Having too many features can be a problem as the solution space becomes much larger and requires more data in order to train the model, otherwise possibly leading to overfitting, known as the curse of dimensionality.